In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.4,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [ ]:
memory = ConversationBufferWindowMemory(
    return_messages=True, memory_key="chat_history", k=5
)


examples = [
    {"input": "Top Gun", "output": "🛩️👨‍✈️🔥"},
    {"input": "The Godfather", "output": "👨‍👨‍👦🔫🍝"},
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("assistant", "{output}"),
    ]
)


fewshot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """당신은 영화 제목을 3개의 이모지로 변환하는 AI입니다.
            - 모든 입력에 대해 반드시 3개의 이모지로만 응답해야 합니다.
            - 이전 대화 내용을 물어보는 경우, chat_history에서 참조해야 합니다.
            - 처음의 두 예시(Top Gun, The Godfather)는 오직 답변 형식을 보여주기 위한 것으로, 실제 답변에 사용하면 안 됩니다.
            """,
        ),
        fewshot_prompt,
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)


chain = LLMChain(llm=llm, prompt=final_prompt, verbose=True, memory=memory)


In [3]:
response1 = chain.invoke({"input": "타짜"})
print(response1["text"])

response2 = chain.invoke({"input": "나는 전설이다"})
print(response2["text"])



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 영화 제목을 3개의 이모지로 변환하는 AI입니다.
            - 모든 입력에 대해 반드시 3개의 이모지로만 응답해야 합니다.
            - 이전 대화 내용을 물어보는 경우, chat_history에서 실제로 언급된 영화만 참조해야 합니다.
            - 아래의 FewShot 예시(Top Gun, The Godfather)는 오직 답변 형식을 보여주기 위한 것으로, 실제 답변에 사용하면 안 됩니다.
            
Human: Top Gun
AI: 🛩️👨‍✈️🔥
Human: The Godfather
AI: 👨‍👨‍👦🔫🍝
Human: 타짜
🃏💰🎲
> Finished chain.
🃏💰🎲


> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 영화 제목을 3개의 이모지로 변환하는 AI입니다.
            - 모든 입력에 대해 반드시 3개의 이모지로만 응답해야 합니다.
            - 이전 대화 내용을 물어보는 경우, chat_history에서 실제로 언급된 영화만 참조해야 합니다.
            - 아래의 FewShot 예시(Top Gun, The Godfather)는 오직 답변 형식을 보여주기 위한 것으로, 실제 답변에 사용하면 안 됩니다.
            
Human: Top Gun
AI: 🛩️👨‍✈️🔥
Human: The Godfather
AI: 👨‍👨‍👦🔫🍝
Human: 타짜
AI: 🃏💰🎲
Human: 나는 전설이다
🧟‍♂️🌆🌅
> Finished chain.
🧟‍♂️🌆🌅


In [4]:
memory_result = chain.invoke({"input": "사용자가 첫 번째로 물어본 영화가 무엇인가요?"})
print(memory_result["text"])



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 영화 제목을 3개의 이모지로 변환하는 AI입니다.
            - 모든 입력에 대해 반드시 3개의 이모지로만 응답해야 합니다.
            - 이전 대화 내용을 물어보는 경우, chat_history에서 실제로 언급된 영화만 참조해야 합니다.
            - 아래의 FewShot 예시(Top Gun, The Godfather)는 오직 답변 형식을 보여주기 위한 것으로, 실제 답변에 사용하면 안 됩니다.
            
Human: Top Gun
AI: 🛩️👨‍✈️🔥
Human: The Godfather
AI: 👨‍👨‍👦🔫🍝
Human: 타짜
AI: 🃏💰🎲
Human: 나는 전설이다
AI: 🧟‍♂️🌆🌅
Human: 사용자가 첫 번째로 물어본 영화가 무엇인가요?
Top Gun
> Finished chain.
Top Gun


In [5]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='타짜'),
  AIMessage(content='🃏💰🎲'),
  HumanMessage(content='나는 전설이다'),
  AIMessage(content='🧟\u200d♂️🌆🌅'),
  HumanMessage(content='사용자가 첫 번째로 물어본 영화가 무엇인가요?'),
  AIMessage(content='Top Gun')]}